In [ ]:
video_results = {} #비디오 데이터를 넣을 dictionary
cnt = 0
for video_id in tqdm(array_video_id): #Video_ID 목록
    if(cnt % 9 == 2):
        time.sleep(3)
    cnt += 1
    result = {} #하나의 row에 담을 dictionary
    video_url = "https://www.youtube.com"+video_id
    response = session.get(video_url,headers = headers) #URL 통신
    if(response.status_code == 429):
        print(response)
    soup = bs(response.text, "html.parser") #beautifulSoup 파싱
    
    try:
        
        data = re.search(r"var ytInitialData = ({.*?});", soup.prettify()).group(1)
        data_json = json.loads(data)
        videoPrimaryInfoRenderer = data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents'][0]['videoPrimaryInfoRenderer']
        videoSecondaryInfoRenderer = data_json['contents']['twoColumnWatchNextResults']['results']['results']['contents'][1]['videoSecondaryInfoRenderer']
        # number of likes
        likes_label = videoPrimaryInfoRenderer['videoActions']['menuRenderer']['topLevelButtons'][0]['toggleButtonRenderer']['defaultText']['accessibility']['accessibilityData']['label'] # "No likes" or "###,### likes"
        like_counts = re.sub(r'[^0-9]', '', likes_label)
        
    except:
        
        like_counts = ''
        
    result['likes'] = like_counts
    
    
    
    html = response.text
    
    try:        
        paidContent = int(str(html).index('paidContentOverlayRenderer'))        
    except:        
        result['ads_yn'] = 0
        paidContent = 0
    try:
        if('유료' in html[paidContent+20:paidContent+100]):
            result['ads_yn'] = 1
    except:
        result['ads_yn'] = 0
    
    try: # 예외 발생하면 가져오지 않음
        meta = soup.find_all("meta") #메타 기본 정보 불러오기
        result['video_id'] = soup.find("meta", itemprop="videoId")['content']
        result['channel_id'] = soup.find("meta", itemprop="channelId")['content']
        result['title'] = soup.find("meta", property="og:title")['content']
        result['image'] = soup.find("meta", property="og:image")['content']
        result['genre'] = soup.find("meta", itemprop="genre")['content']
        result['published_date'] = soup.find("meta", itemprop="datePublished")['content']
        view_count = soup.find("meta", itemprop="interactionCount")
        if(view_count != None):
            result['views'] = soup.find("meta", itemprop="interactionCount")['content']
        else:
            result['views'] = 0
        result['duration'] =  soup.find("meta", itemprop="duration")['content']
        for tag in meta:
            if 'name' in tag.attrs.keys() and tag.attrs['name'].strip().lower() in ['description', 'keywords']:
                result[tag.attrs['name']] = tag.attrs['content']
        video_results[video_id] = result
    except:
        continue  